Maybe useful

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
import pydicom
from glob import glob
from tqdm.notebook import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
# import matplotlib.pyplot as plt
# from skimage import exposure
# import cv2
import warnings
warnings.filterwarnings('ignore')

### ↓Refer [here](https://www.kaggle.com/trungthanhnguyen0502/eda-vinbigdata-chest-x-ray-abnormalities) 

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data 

In [ ]:
def get_width_height(dicom_paths):
    w_h = pd.DataFrame(columns=['id', 'height', 'width'])
    for path in tqdm(dicom_paths):
        img = dicom2array(path)
        temp = path[61:93]
        temp_df = {'id':temp , 'height':img.shape[0] , 'width':img.shape[1]}
#         print(temp_df)
        w_h = w_h.append(temp_df , ignore_index=True)
#    print(w_h)
    return w_h

In [ ]:
dataset_dir = '../input/vinbigdata-chest-xray-abnormalities-detection'
dicom_paths = glob(f'{dataset_dir}/train/*.dicom')
#dicom_paths[0][61:93]  # 从文件夹信息获取片子的id

### ↓Take about **`5.5`** hours.

In [ ]:
%%time
# w = get_width_height(dicom_paths[:15])
w = get_width_height(dicom_paths)
w.head(10)

In [ ]:
w.shape

In [ ]:
w.to_csv('./width_height.csv')

In [ ]:
s = pd.read_csv('./width_height.csv',index_col = 0)

In [ ]:
s.shape

## But this way to get width and height maybe cause loss of information from **`.dicom`**.
## To be verified...